In [1]:
import GetPropertiesAPI as GP
import BuildConsolidatedFeaturesFile as BCF
import importlib
importlib.reload(GP)
importlib.reload(BCF)
import csv
import json
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium import plugins
import statistics as s

In [2]:
BCF.buildExifFeatureFl("../data/all_imgs_expt2.csv","../data/imgs_exif_data_expt2.json")

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
with open("../data/ggr_gid_uuid_exif_ftr_map.json","r") as inFl:
    imgProps = json.load(inFl)

In [ ]:
gidExifDf = pd.DataFrame(imgProps).transpose()
gidExifDf.reset_index(inplace=True)
gidExifDf.columns = ['GID',"contributor", 'datetime', 'lat','long' ]
# gidExifDf[['GID']] = gidExifDf[['GID']].apply(pd.to_numeric)

In [ ]:
with open("../FinalResults/rankListImages_expt2.csv","r") as inFl:
    gidResultsDf = pd.DataFrame.from_csv(inFl)
    
gidResultsDf.reset_index(inplace=True)

In [ ]:
df = pd.merge(gidExifDf,gidResultsDf,left_on='GID',right_on='GID')

In [7]:
d = df.to_dict()['Proportion']
bin_shr = {}
for key in d.keys():
    if d[key] <= 20:
        bin_shr[key] = 0
    elif d[key] >= 80:
        bin_shr[key] = 1
        
D = df.to_dict()
D['bin_shr'] = bin_shr
df = pd.DataFrame.from_dict(D)

unshared = df[(df['bin_shr'] == 0)]
unshared = unshared[['lat','long']]

shared = df[(df['bin_shr'] == 1)]
shared = shared[['lat','long']]

In [35]:
shared_lats = list(shared['lat'])
shared_longs = list(shared['long'])

map_loc = folium.Map(location=[-1.369212, 36.848628],tiles='Stamen Terrain',zoom_start=12)

for i in range(0,len(shared_lats)):
    folium.Marker([shared_lats[i],shared_longs[i]],
              icon=folium.Icon(color='green',icon='info-sign')
              ).add_to(map_loc)

unshared_lats = list(unshared['lat'])
unshared_longs = list(unshared['long'])

for i in range(0,len(unshared_lats)):
    folium.Marker([unshared_lats[i],unshared_longs[i]],
              icon=folium.Icon(color='red',icon='info-sign')
              ).add_to(map_loc)
    

map_loc.save(outfile='../FinalResults/ClickLocations.html')

In [34]:
# Shared Data Clusters
cluster_map = folium.Map(location=[-1.369212, 36.848628],tiles='Stamen Terrain',zoom_start=11)
marker_cluster = folium.MarkerCluster("Shares").add_to(cluster_map)

for i in range(0,len(shared_lats)):
    folium.Marker([shared_lats[i],shared_longs[i]],
                  icon=folium.Icon(color='green',icon='info-sign'),
                  popup="Shared"
                 ).add_to(marker_cluster)
    
cluster_map.save(outfile='../FinalResults/ShareLocationClusters.html')

# Not Shared Data Clusters
cluster_map2 = folium.Map(location=[-1.369212, 36.848628],tiles='Stamen Terrain',zoom_start=11)
marker_cluster2 = folium.MarkerCluster("Not Shares").add_to(cluster_map2)
for i in range(0,len(unshared_lats)):
    folium.Marker([unshared_lats[i],unshared_longs[i]],
                 icon=folium.Icon(color='red',icon='info-sign'),
                  popup="Not Shared"
                 ).add_to(marker_cluster2)
    

cluster_map2.save(outfile='../FinalResults/NotShareLocationClusters.html')

In [8]:
a,b = list(shared.lat),list(shared.long)
c,d = list(unshared.lat),list(unshared.long)
locs_shared = [[a[i],b[i]] for i in range(len(a))]
locs_unshared = [[c[i],d[i]] for i in range(len(c))]

In [32]:
heatmap_map = folium.Map(location=[-1.369212, 36.848628],tiles='Stamen Terrain', zoom_start=12)
hm = plugins.HeatMap(locs_shared)
heatmap_map.add_children(hm)

heatmap_map.save("../FinalResults/heatMap_Shared.html")

In [33]:
heatmap_map = folium.Map(location=[-1.369212, 36.848628],tiles='Stamen Terrain', zoom_start=12)
hm = plugins.HeatMap(locs_unshared)
heatmap_map.add_children(hm)

heatmap_map.save("../FinalResults/heatMap_NotShared.html")

In [9]:
gidExifDf.head()

,GID,contributor,datetime,lat,long
0,00035379-6f25-15e1-b984-7a915ae80a2a,"GGR,50,A,00220",2016-01-30 14:23:39,0.456715,36.7774
1,0004a0fc-157d-a44e-9fc0-8ef61a7c756c,"GGR,75,D,01268",2016-02-01 10:14:15,-1,-1
2,0004b3f7-6fe7-4fd3-4dcb-bd180b6da607,"GGR,113,A,00123",2016-01-30 13:06:01,0.677527,36.6032
3,0006f0c6-1dfe-a73c-ec62-a613ce1a1ea7,"GGR,20,A,00151",2016-01-31 15:59:07,0.29773,36.934
4,00074c81-95b2-1d8e-f428-decf62881e8c,"GGR,15,A,00242",2016-01-30 09:47:02,0.44139,36.9364


In [17]:
lat_long = [(lats[i], longs[i]) for i in range(len(gidExifDf)) if lats[i] != -1 and longs[i] != -1]

In [22]:
heatmap_map = folium.Map(location=[-1.369212, 36.848628],tiles='Stamen Terrain', zoom_start=12)
hm = plugins.HeatMap(lat_long)
heatmap_map.add_children(hm)

heatmap_map.save("/tmp/heatMap_GGR.html")